In [6]:
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [5]:
cap = cv2.VideoCapture(0)
cv2.namedWindow('Mediapipe Feed', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Mediapipe Feed', 1920, 1080)  # Set desired window size
counter = 0

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose: #confidence level set-up
    while cap.isOpened():
        ret, frame = cap.read()

        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) #to recolour img
        image.flags.writeable = False 

        results = pose.process(image)    #for detection

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)


        #to calculate angle
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            
            r_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER]
            r_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW]
            r_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST]
            r_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP]
            r_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE]
            r_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE]
            

            l_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER]
            l_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW]
            l_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST]
            l_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP]
            l_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE]
            l_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE]
            

            s_r = np.array([r_shoulder.x, r_shoulder.y])
            e_r = np.array([r_elbow.x, r_elbow.y])
            w_r = np.array([r_wrist.x, r_wrist.y])
            h_r = np.array([r_hip.x, r_hip.y])
            k_r = np.array([r_knee.x, r_knee.y])
            a_r = np.array([r_ankle.x, r_ankle.y])
            
            s_l = np.array([l_shoulder.x, l_shoulder.y])
            e_l = np.array([l_elbow.x, l_elbow.y])
            w_l = np.array([l_wrist.x, l_wrist.y])
            h_l = np.array([l_hip.x, l_hip.y])
            k_l = np.array([l_knee.x, l_knee.y])
            a_l = np.array([l_ankle.x, l_ankle.y])

            

            def angle_calc(a,b,c):
                ba = a - b
                bc = c - b
                cos_theta = np.dot(ba,bc)/(np.linalg.norm(ba)*np.linalg.norm(bc))
                angle = np.arccos(np.clip(cos_theta, -1.0, 1.0))
                return np.degrees(angle)
                

            r_elbow_angle = angle_calc(s_r,e_r,w_r)
            l_elbow_angle = angle_calc(s_l,e_l,w_l)

            r_ap_angle = angle_calc(h_r,s_r,e_r)
            l_ap_angle = angle_calc(h_l,s_l,e_l)

            r_hip_angle = angle_calc(h_l, h_r, k_r)
            l_hip_angle = angle_calc(h_r, h_l, k_l)

            r_knee_angle = angle_calc(h_r, k_r, a_r)
            l_knee_angle = angle_calc(h_l, k_l, a_l)

            
            coordinates_elbow_r = np.multiply([e_r[0],e_r[1]], [image.shape[1], image.shape[0]]).astype(int)
            coordinates_elbow_l = np.multiply([e_l[0],e_l[1]], [image.shape[1], image.shape[0]]).astype(int)
            
            coordinates_ap_r = np.multiply([s_r[0],s_r[1]], [image.shape[1], image.shape[0]]).astype(int)
            coordinates_ap_l = np.multiply([s_l[0],s_l[1]], [image.shape[1], image.shape[0]]).astype(int)

            coordinates_hip_r = np.multiply([h_r[0],h_r[1]], [image.shape[1], image.shape[0]]).astype(int)
            coordinates_hip_l = np.multiply([h_l[0],h_l[1]], [image.shape[1], image.shape[0]]).astype(int)

            coordinates_knee_r = np.multiply([k_r[0],k_r[1]], [image.shape[1], image.shape[0]]).astype(int)
            coordinates_knee_l = np.multiply([k_l[0],k_l[1]], [image.shape[1], image.shape[0]]).astype(int)            

            cv2.putText(image, str(int(r_elbow_angle)),tuple(coordinates_elbow_r), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
            cv2.putText(image, str(int(l_elbow_angle)),tuple(coordinates_elbow_l), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
            
            cv2.putText(image, str(int(r_ap_angle)),tuple(coordinates_ap_r), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
            cv2.putText(image, str(int(l_ap_angle)),tuple(coordinates_ap_l), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)

            cv2.putText(image, str(int(r_hip_angle)),tuple(coordinates_hip_r), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
            cv2.putText(image, str(int(l_hip_angle)),tuple(coordinates_hip_l), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)

            cv2.putText(image, str(int(r_knee_angle)),tuple(coordinates_knee_r), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
            cv2.putText(image, str(int(l_knee_angle)),tuple(coordinates_knee_l), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)

    
            if r_elbow_angle > 160 and l_elbow_angle > 160:
                pos = "down"
            if r_elbow_angle < 30 and l_elbow_angle < 30 and pos == 'down':
                pos = "up"
                counter += 1
                print(counter)
                

        
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            
        

        image = cv2.resize(image, (1920, 1080))
        cv2.imshow('Mediapipe Feed', image) #loop through the frames
      
        if cv2.waitKey(10) & 0xFF == ord('q'): #to close the window
            break
    
    cap.release()
    cv2.destroyAllWindows()

I0000 00:00:1751622011.185668    6853 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1751622011.189684    7037 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.2.8-1ubuntu1~24.04.1), renderer: Mesa Intel(R) Graphics (ADL GT2)
W0000 00:00:1751622011.286313    7029 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751622011.326851    7031 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


1
2
3
